In [1]:
import pandas as pd
import pyodbc
from datamart import fix_me
import useful_functions as use

In [2]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Feb 19 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
oncology = ['PRD','ON','HO','GO','OMO','PHO','RO','ASO','SO']
oncologists = ppd[ppd.PRIM_SPEC_CD.isin(oncology)]
len(oncologists)

33487

In [4]:
oncologists['IQVIA_ME'] = [x[0:10] for x in oncologists.ME]
oncology_me = tuple(oncologists.IQVIA_ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
#iqvia
username = 'vigrose'
password = 'Hufflepuff~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        f"""
        SELECT
        B.BUSINESS_NAME,
        B.COT_CLASSIFICATION,
        B.OWNER_STATUS,
        A.AFFIL_IND,
        P.ME
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.ME IN {oncology_me}
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,BUSINESS_NAME,COT_CLASSIFICATION,OWNER_STATUS,AFFIL_IND,ME
0,MEDICAL SOCIETY OF THE STATE OF NEW YORK,Insurance,INDEPENDENT,A,0354697103
1,MEDICAL SOCIETY OF THE STATE OF NEW YORK,Insurance,INDEPENDENT,A,0354697103
2,MEDICAL SOCIETY OF THE STATE OF NEW YORK,Insurance,INDEPENDENT,A,0354697103
3,MEDICAL SOCIETY OF THE STATE OF NEW YORK,Insurance,INDEPENDENT,A,0354697103
4,MEDICAL SOCIETY OF THE STATE OF NEW YORK,Insurance,INDEPENDENT,A,0354697103


In [16]:
indie_onc = pd.merge(oncologists, indies, left_on='IQVIA_ME',right_on='ME')

In [14]:
indies = iqvia[iqvia.OWNER_STATUS=='INDEPENDENT'].drop_duplicates()

In [17]:
indie_onc[indie_onc.AFFIL_IND=='P']

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,COT_CLASSIFICATION,OWNER_STATUS,AFFIL_IND,ME_y
0,00102000374,A,1,2.0,SCOTT A MC DANIEL MD,MC DANIEL,SCOTT,ANDREW,NaN,NaN,...,NaN,NaN,NaN,NaN,0010200037,"JACKSON HOSPITAL AND CLINIC, INC",Hospital,INDEPENDENT,P,0010200037
4,00102040031,A,1,1.0,CHRISTINE C OLSEN MD,OLSEN,CHRISTINE,CECELIA,NaN,MASS GEN HOSP,...,NaN,NaN,NaN,NaN,0010204003,HARVARD DERMATOLOGY RESIDENCY TRAINING PROGRAM,Academic,INDEPENDENT,P,0010204003
5,00102050771,A,1,1.0,JOE S LILES MD,LILES,JOE,SPENCER,NaN,UNIV OF SOUTH ALABAMA DEPT OF SURGERY,...,NaN,NaN,NaN,NaN,0010205077,SPRINGHILL MEDICAL CENTER,Hospital,INDEPENDENT,P,0010205077
9,00102080637,A,1,1.0,LINDSEY M MARTIN MD,MARTIN,LINDSEY,MARIE,NaN,VANDERBILT,...,NaN,NaN,NaN,NaN,0010208063,WILLIAMSON COUNTY HOSPITAL DISTRICT,Hospital,INDEPENDENT,P,0010208063
10,00102080637,A,1,1.0,LINDSEY M MARTIN MD,MARTIN,LINDSEY,MARIE,NaN,VANDERBILT,...,NaN,NaN,NaN,NaN,0010208063,NORTHCREST MEDICAL CENTER,Hospital,INDEPENDENT,P,0010208063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8023,95701970099,A,1,1.0,MARIJO BILUSIC MD,BILUSIC,MARIJO,NaN,NaN,FOX CHASE CANCER CENTER,...,NaN,NaN,NaN,NaN,9570197009,SYLVESTER COMPREHENSIVE CANCER CENTERS ALEXS P...,Outpatient Center,INDEPENDENT,P,9570197009
8027,95702930015,A,1,2.0,NENAD ANTIC MD,ANTIC,NENAD,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9570293001,BEVERLY COMMUNITY HOSPITAL ASSOCIATION,Hospital,INDEPENDENT,P,9570293001
8028,95702980080,A,1,1.0,SINISA STANIC MD,STANIC,SINISA,NaN,NaN,RADIATION ONCOLOGY,...,SINISA,NaN,NaN,NaN,9570298008,UNIVERSITY OF ILLINOIS COLLEGE OF MEDICINE AT ...,Academic,INDEPENDENT,P,9570298008
8031,96501770032,A,1,1.0,LATHA SUBRAMANIAN MD,SUBRAMANIAN,LATHA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9650177003,"ANCHORAGE ONCOLOGY CENTER, LLC",Outpatient Center,INDEPENDENT,P,9650177003


In [27]:
indie_onc

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,COT_CLASSIFICATION,OWNER_STATUS,AFFIL_IND,ME_y
0,00102000374,A,1,2.0,SCOTT A MC DANIEL MD,MC DANIEL,SCOTT,ANDREW,NaN,NaN,...,NaN,NaN,NaN,NaN,0010200037,"JACKSON HOSPITAL AND CLINIC, INC",Hospital,INDEPENDENT,P,0010200037
1,00102010647,A,1,2.0,ALYSSA G RIEBER MD,RIEBER,ALYSSA,GOSNEY,NaN,NaN,...,ALYSSA,NaN,NaN,NaN,0010201064,UPLAND MEDICAL GROUP,Insurance,INDEPENDENT,A,0010201064
2,00102020154,A,1,1.0,HEJAL C PATEL MD,PATEL,HEJAL,CHANDRAKANT,NaN,NaN,...,NaN,NaN,NaN,NaN,0010202015,MIZELL MEMORIAL HOSPITAL,Hospital,INDEPENDENT,A,0010202015
3,00102021673,A,1,2.0,GAUTAM PRASAD MD,PRASAD,GAUTAM,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0010202167,BROWN AND TOLAND PHYSICIANS,Insurance,INDEPENDENT,A,0010202167
4,00102040031,A,1,1.0,CHRISTINE C OLSEN MD,OLSEN,CHRISTINE,CECELIA,NaN,MASS GEN HOSP,...,NaN,NaN,NaN,NaN,0010204003,HARVARD DERMATOLOGY RESIDENCY TRAINING PROGRAM,Academic,INDEPENDENT,P,0010204003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,95702980080,A,1,1.0,SINISA STANIC MD,STANIC,SINISA,NaN,NaN,RADIATION ONCOLOGY,...,SINISA,NaN,NaN,NaN,9570298008,UNIVERSITY OF ILLINOIS COLLEGE OF MEDICINE AT ...,Academic,INDEPENDENT,P,9570298008
8029,95707880016,A,1,1.0,SINISA DOVAT MD,DOVAT,SINISA,NaN,NaN,PEDIATRICS-HEMATOLOGY/ONCOLOGY,...,NaN,NaN,NaN,NaN,9570788001,PENNSYLVANIA STATE UNIVERSITY COLLEGE OF MEDICINE,Academic,INDEPENDENT,A,9570788001
8030,96501770032,A,1,1.0,LATHA SUBRAMANIAN MD,SUBRAMANIAN,LATHA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9650177003,PROVIDENCE HEALTH AND SERVICES ALASKA,Insurance,INDEPENDENT,A,9650177003
8031,96501770032,A,1,1.0,LATHA SUBRAMANIAN MD,SUBRAMANIAN,LATHA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9650177003,"ANCHORAGE ONCOLOGY CENTER, LLC",Outpatient Center,INDEPENDENT,P,9650177003


In [26]:
indie_onc.groupby('COT_CLASSIFICATION').count()

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,OWNER_STATUS,AFFIL_IND,ME_y
COT_CLASSIFICATION,,,,,,,,,,,,,,,,,,,,,
Academic,902,902,902,883,902,902,902,669,17,371,...,115,154,14,8,14,902,902,902,902,902
Correctional Facility,6,6,6,6,6,6,6,5,1,1,...,2,3,0,0,0,6,6,6,6,6
Hospital,1395,1395,1395,1367,1395,1395,1395,1083,55,337,...,271,348,45,28,45,1395,1395,1395,1395,1395
Insurance,2196,2196,2196,2187,2196,2196,2196,1635,28,421,...,370,480,65,50,65,2196,2196,2196,2196,2196
Laboratory,2,2,2,2,2,2,2,0,0,2,...,0,0,0,0,0,2,2,2,2,2
Miscellaneous,242,242,242,238,242,242,242,177,5,68,...,29,31,4,4,4,242,242,242,242,242
Outpatient Center,2960,2960,2960,2916,2960,2960,2960,2263,74,615,...,602,752,89,64,89,2960,2960,2960,2960,2960
Pharmacy,6,6,6,6,6,6,6,6,1,2,...,2,2,0,0,0,6,6,6,6,6
Research,134,134,134,132,134,134,134,109,3,39,...,20,23,2,2,2,134,134,134,134,134


In [25]:
indie_onc[['ME_x', 'PRIM_SPEC_CD', 'BUSINESS_NAME',
       'COT_CLASSIFICATION', 'OWNER_STATUS', 'AFFIL_IND']].rename(columns={'ME_x':'ME'}).to_csv('../../Data/Asks/Independent_Oncologists_220223.csv', index=False)

In [ ]:
['ME_x', 'MAILING_NAME', 'PRIM_SPEC_CD', 'BUSINESS_NAME',
       'COT_CLASSIFICATION', 'OWNER_STATUS', 'AFFIL_IND']